In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [11]:
train = pd.read_csv('../../dataset/train.csv')
valid = pd.read_csv('../../dataset/val.csv')
test = pd.read_csv('../../dataset/test.csv')

In [12]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 168

In [13]:
xtrain, ytrain = train.Comment.values, train.Emotion.values
xvalid, yvalid = valid.Comment.values, valid.Emotion.values
xtest, ytest = test.Comment.values, test.Emotion.values
                                                  

train['Comment'].apply(lambda x:len(str(x).split())).max()

168

In [14]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 168

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtest_seq = token.texts_to_sequences(xtest)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [15]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [16]:
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 168, 300)          4167000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               40100     
_________________________________________________________________
dense (Dense)                (None, 7)                 707       
Total params: 4,207,807
Trainable params: 4,207,807
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(xtrain_pad, ytrain,epochs=10,validation_data=(xvalid_pad, yvalid), batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/10
81/81 [==============================] - 10s 123ms/step - loss: 1.9362 - accuracy: 0.1813 - val_loss: 1.9342 - val_accuracy: 0.1966
Epoch 2/10
81/81 [==============================] - 11s 135ms/step - loss: 1.7543 - accuracy: 0.5486 - val_loss: 1.9072 - val_accuracy: 0.2111
Epoch 3/10
81/81 [==============================] - 11s 140ms/step - loss: 1.4187 - accuracy: 0.8285 - val_loss: 1.8999 - val_accuracy: 0.2340
Epoch 4/10
81/81 [==============================] - 11s 141ms/step - loss: 1.2670 - accuracy: 0.9300 - val_loss: 1.9057 - val_accuracy: 0.2068
Epoch 5/10
81/81 [==============================] - 11s 140ms/step - loss: 1.2132 - accuracy: 0.9653 - val_loss: 1.8988 - val_accuracy: 0.2272
Epoch 6/10
81/81 [==============================] - 12s 145ms/step - loss: 1.1926 - accuracy: 0.9799 - val_loss: 1.9023 - val_accuracy: 0.2323
Epoch 7/10
81/81 [==============================] - 11s 133ms/step - loss: 1.1841 - accuracy: 0.9855 - val_loss: 1.8969 - val_accuracy: 0.2417

In [18]:
model.evaluate(xtest_pad, ytest)

37/37 [==============================] - 1s 18ms/step - loss: 1.9066 - accuracy: 0.2178


[1.9065964221954346, 0.2178046703338623]